## SARCASM DETECTION
### Gabrielle Marie Angeles, Rachel Hydie Martinez

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.13.1


In [5]:
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)

In [7]:
data = data.drop(['article_link'], axis=1)
data.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [8]:
data['is_sarcastic'].value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

In [9]:
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 21367
Test size: 5342


In [10]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [11]:
train_sarc, test_sarc = train_test_split(data['is_sarcastic'], train_size)
train_headline, test_headline = train_test_split(data['headline'], train_size)

In [12]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

In [14]:
tokenize.fit_on_texts(train_headline) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_headline)
x_test = tokenize.texts_to_matrix(test_headline)

In [19]:
# Converts the labels to a one-hot representation
num_classes = np.max(train_sarc) + 1
y_train = keras.utils.to_categorical(train_sarc, num_classes)
y_test = keras.utils.to_categorical(test_sarc, num_classes)

In [20]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (21367, 1000)
x_test shape: (5342, 1000)
y_train shape: (21367, 2)
y_test shape: (5342, 2)


In [21]:
batch_size = 32
epochs = 2
drop_ratio = 0.5

In [22]:
# Build the model
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [23]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 19230 samples, validate on 2137 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
19230/19230 [==============================] - 7s 359us/sample - loss: 0.4335 - acc: 0.7912 - val_loss: 0.3975 - val_acc: 0.8184
Epoch 2/2
19230/19230 [==============================] - 6s 297us/sample - loss: 0.3405 - acc: 0.8471 - val_loss: 0.3850 - val_acc: 0.8306


In [24]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5342/5342 [==============================] - 0s 80us/sample - loss: 0.4053 - acc: 0.8137
Test loss: 0.40532066415017665
Test accuracy: 0.8137402


In [36]:
encoder = LabelEncoder()
encoder.fit(train_sarc)
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_headline.iloc[i][:50], "...")
    print('Actual label:' + str(test_sarc.iloc[i]))
    print("Predicted label: " + str(predicted_label) + "\n")  

report: 98% of german sexual intercourse uploaded  ...
Actual label:1
Predicted label: 1

video store's 'favorites' shelf offers telling gli ...
Actual label:1
Predicted label: 0

u.s. encouraging cuba to shift toward democratic s ...
Actual label:1
Predicted label: 0

teen learns the negligible value of a dollar ...
Actual label:1
Predicted label: 0

are you a 'good' influence on your kids? ...
Actual label:0
Predicted label: 0

tv commercial for beverage features woefully reckl ...
Actual label:1
Predicted label: 1

the gentrification of higher ed ...
Actual label:0
Predicted label: 0

nation's women clarify they harbor no secret desir ...
Actual label:1
Predicted label: 1

man brings visiting parents into office to meet co ...
Actual label:1
Predicted label: 1

tsa agent can't bring himself to make dad take off ...
Actual label:1
Predicted label: 0

